In [1]:
import numpy as np
import xarray as xr
from datetime import datetime
import json
from pystac_client import Client

In [12]:
# which collections to search
url='https://cmr.earthdata.nasa.gov/stac/LPCLOUD'
collections = ['HLSL30.v2.0', 'HLSS30.v2.0']

# define the dates we want to query
start_date = datetime(year=2018,month=9,day=1)
end_date = datetime(year=2019,month=9,day=30)

# based on the lidar tiles
domains_direc = '/Users/jpflug/Documents/Projects/snowCover_comparisons/Data/ASO/selected/'
domain_id = 7
trial_id = 33

# # MODIS data direc
# mod_direcc = '/Users/jpflug/Documents/Projects/snowCover_comparisons/Data/MODIS/'

# HLS out data directory
hls_out = '/Users/jpflug/Documents/Projects/snowCover_comparisons/Data/HLS/'

In [13]:
dt_min = start_date.strftime("%Y-%m-%d")
dt_max = end_date.strftime("%Y-%m-%d")
lim=1000


# fname = 'domain'+str(domain_id).zfill(2)+'_trial'+str(trial_id).zfill(2)+'_2019_MOD10Av61.nc'
fname = 'domain'+str(domain_id).zfill(2)+'_trial'+str(trial_id).zfill(2)+'.tif'
ds = xr.open_dataset(domains_direc+fname)
ds = ds.rio.reproject(4326)
pt = {"type":"Point", "coordinates":[float(ds['x'].mean()),float(ds['y'].mean())]}
pt = json.dumps(pt)
print(pt)

# open the catalog
catalog = Client.open(f'{url}')

search = catalog.search(
    collections=collections,
    intersects=pt,
    datetime=dt_min + '/' + dt_max,
    limit=lim
)

print('Found', search.matched(), 'granules at point', pt, 'from', dt_min, 'to', dt_max)

# Initialize an empty list to store tile IDs
tile_ids = []

# Iterate through paginated results
for counter,item in enumerate(search.get_items()):
#     print(item)
    try:
        tile_ids.append(item.id.split('.')[2][1:])
    except:
        print('exception',item)
    if counter > 20:
        break

print(set(tile_ids))

{"type": "Point", "coordinates": [-106.5475763902401, 37.31518996673092]}
Found 94 granules at point {"type": "Point", "coordinates": [-106.5475763902401, 37.31518996673092]} from 2018-09-01 to 2019-09-30


/Users/jpflug/opt/anaconda3/envs/pangeo/lib/python3.8/site-packages/pystac_client/item_search.py:834: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


{'13SCB'}


In [3]:
# # CO_select_1 = [29,4,25,2,6,19,33,3,23,18,11,13]
# # CO_select_2 = [92,51,39,94,65,1,36,95,18,52,1,77]
# # reducing based on the bad domains in CFL_select1
# CO_select_1 = [29,4,25,2,6,33,3,23]
# CO_select_2 = [92,51,39,94,65,36,95,18]
# CFL_select_1 = [29,4,25,2,6,33,3,23]
# CFL_select_2 = [77,82,63,85,87,2,79,15]
# CFH_select_1 = [29,4,25,2,6,33,3,23]
# CFH_select_2 = [42,98,10,12,16,6,87,99]

# # SO_select_1 = [16,10,14,5,32,35,7,27,9,12,28,15]
# # SO_select_2 = [1,9,25,40,29,54,85,54,41,54,45,83]
# # reducing based on bad domains in SFL_select1
# SO_select_1 = [16,10,14,5,32,35,7,27,9,12,15]
# SO_select_2 = [1,9,25,40,29,54,85,54,41,54,83]
# SFL_select_1 = [16,10,14,5,32,35,7,27,9,12,15]
# SFL_select_2 = [71,40,72,44,73,55,54,9,49,18,76]
# SFH_select_1 = [16,10,14,5,32,35,7,27,9,12,15]
# SFH_select_2 = [48,1,16,10,26,5,33,95,91,57,25]

In [4]:
def search_stac_for_HLS(pt, dt_min, dt_max, cloudcover_max=80, lim=100, url='https://cmr.earthdata.nasa.gov/stac/LPCLOUD', collections=['HLSL30_2.0', 'HLSS30_2.0']):
    # open the catalog
    catalog = Client.open(f'{url}')
    
    # perform the search
    search = catalog.search(
        collections=collections,
        intersects=pt,
        datetime=dt_min + '/' + dt_max,
        limit=lim
    )

    links = []

    if search.matched() == 0:
        print('No granules found at point', pt, 'from', dt_min, 'to', dt_max)
    else:
        print('Found', search.matched(), 'granules at point', pt, 'from', dt_min, 'to', dt_max)
        item_collection = search.get_all_items()

    return item_collection

def indentify_tiles(d1,d2,tiles,domains,trials):
    for s1,s2 in zip(d1,d2):
        fname = 'domain'+str(s1).zfill(2)+'_trial'+str(s2).zfill(2)+'_2019_MOD10Av61.nc'
        ds = xr.open_dataset(mod_direcc+fname)

        pt = {"type":"Point", "coordinates":[float(ds['lon'].mean()),float(ds['lat'].mean())]}
        pt = json.dumps(pt)
        
        items = search_stac_for_HLS(pt, start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))
        tiles.append(items[0].id.split('.')[2][1:])
        domains.append(s1)
        trials.append(s2)
    return tiles,domains,trials

# # temporary code attempt. previous code is included above
# def search_stac_for_HLS(pt, dt_min, dt_max, cloudcover_max=80, lim=100, url='https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/', collections=['HLSL30_2.0', 'HLSS30_2.0']):
#     # open the catalog
#     catalog = Client.open(f'{url}')
    
#     # perform the search
#     search = catalog.search(
#         collections=collections,
#         intersects=pt,  # This can now be a polygon
#         datetime=dt_min + '/' + dt_max,
#         limit=lim
#     )

#     links = []
#     item_collection = None

#     if search.matched() == 0:
#         print('No granules found for area', pt, 'from', dt_min, 'to', dt_max)
#     else:
#         print('Found', search.matched(), 'granules for area', pt, 'from', dt_min, 'to', dt_max)
#         item_collection = search.get_all_items()

#     return item_collection

# def identify_tiles(d1, d2, tiles, domains, trials):
#     for s1, s2 in zip(d1, d2):
#         fname = 'domain'+str(s1).zfill(2)+'_trial'+str(s2).zfill(2)+'_2019_MOD10Av61.nc'
#         ds = xr.open_dataset(mod_direcc+fname)
        
#         # Get the bounding box coordinates from the xarray dataset
#         min_lon, max_lon = ds['lon'].min().item(), ds['lon'].max().item()
#         min_lat, max_lat = ds['lat'].min().item(), ds['lat'].max().item()

#         # Create a polygon representing the bounding box
#         pt = {
#             "type": "Polygon",
#             "coordinates": [[
#                 [min_lon, min_lat],  # Lower-left corner
#                 [min_lon, max_lat],  # Upper-left corner
#                 [max_lon, max_lat],  # Upper-right corner
#                 [max_lon, min_lat],  # Lower-right corner
#                 [min_lon, min_lat]   # Closing the polygon
#             ]]
#         }
#         pt = json.dumps(pt)

#         items = search_stac_for_HLS(pt, start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))
#         if items:
#             tiles.append(items[0].id.split('.')[2][1:])
#             domains.append(s1)
#             trials.append(s2)
#     return tiles, domains, trials

def write_txtFile(fname,strings_list):
    with open(fname,'w') as file:
        for item in strings_list:
            file.write(f"{item}\n")
    return None

In [20]:
dt_min = start_date.strftime("%Y-%m-%d")
dt_max = end_date.strftime("%Y-%m-%d")
lim=100


# fname = 'domain'+str(domain_id).zfill(2)+'_trial'+str(trial_id).zfill(2)+'_2019_MOD10Av61.nc'
fname = 'domain'+str(domain_id).zfill(2)+'_trial'+str(trial_id).zfill(2)+'.tif'
ds = xr.open_dataset(domains_direc+fname)
ds = ds.rio.reproject(4326)
pt = {"type":"Point", "coordinates":[float(ds['x'].mean()),float(ds['y'].mean())]}
pt = json.dumps(pt)
print(pt)

# open the catalog
catalog = Client.open(f'{url}')

search = catalog.search(
    collections=collections,
    intersects=pt,
    datetime=dt_min + '/' + dt_max,
    limit=lim
)

print('Found', search.matched(), 'granules at point', pt, 'from', dt_min, 'to', dt_max)

# Initialize an empty list to store tile IDs
tile_ids = []

# Iterate through paginated results
for counter,item in enumerate(search.get_items()):
#     print(item)
    try:
        tile_ids.append(item.id.split('.')[2][1:])
    except:
        print('exception',item)
    if counter > 20:
        break

print(set(tile_ids))

{"type": "Point", "coordinates": [-120.91156885940596, 40.987461118918134]}
Found 90 granules at point {"type": "Point", "coordinates": [-120.91156885940596, 40.987461118918134]} from 2018-09-01 to 2019-09-30


In [46]:
# Initialize an empty list to store tile IDs
tile_ids = []

# Iterate through paginated results
for counter,item in enumerate(search.get_items()):
#     print(item)
    try:
        tile_ids.append(item.id.split('.')[2][1:])
    except:
        print('exception',item)
    if counter > 20:
        break

print(set(tile_ids))

<Item id=HLS.S30.T10TFL.2018245T184921.v2.0>
<Item id=HLS.L30.T10TFL.2018248T184427.v2.0>
<Item id=HLS.S30.T10TFL.2018255T184951.v2.0>
<Item id=HLS.S30.T10TFL.2018260T185019.v2.0>
<Item id=HLS.L30.T10TFL.2018264T184432.v2.0>
<Item id=HLS.S30.T10TFL.2018265T185101.v2.0>
<Item id=HLS.S30.T10TFL.2018270T185129.v2.0>
<Item id=HLS.S30.T10TFL.2018275T185211.v2.0>
<Item id=HLS.L30.T10TFL.2018280T184440.v2.0>
<Item id=HLS.S30.T10TFL.2018280T185249.v2.0>
<Item id=HLS.S30.T10TFL.2018285T185321.v2.0>
<Item id=HLS.S30.T10TFL.2018290T185359.v2.0>
<Item id=HLS.S30.T10TFL.2018295T185421.v2.0>
<Item id=HLS.L30.T10TFL.2018296T184446.v2.0>
<Item id=HLS.S30.T10TFL.2018300T185459.v2.0>
<Item id=HLS.S30.T10TFL.2018305T185521.v2.0>
<Item id=HLS.S30.T10TFL.2018310T185559.v2.0>
<Item id=HLS.L30.T10TFL.2018312T184448.v2.0>
<Item id=HLS.S30.T10TFL.2018315T185621.v2.0>
<Item id=HLS.S30.T10TFL.2018320T185639.v2.0>
<Item id=HLS.S30.T10TFL.2018325T185701.v2.0>
<Item id=HLS.L30.T10TFL.2018328T184448.v2.0>
{'10TFL'}


In [45]:
counter

89

In [60]:
tiles_running = []
domains_running = []
trials_running =[]

tiles_running,domains_running,trials_running = indentify_tiles(CO_select_1,CO_select_2,tiles_running,domains_running,trials_running)
tiles_running,domains_running,trials_running = indentify_tiles(CFL_select_1,CFL_select_2,tiles_running,domains_running,trials_running)
tiles_running,domains_running,trials_running = indentify_tiles(CFH_select_1,CFH_select_2,tiles_running,domains_running,trials_running)
tiles_running,domains_running,trials_running = indentify_tiles(SO_select_1,SO_select_2,tiles_running,domains_running,trials_running)
tiles_running,domains_running,trials_running = indentify_tiles(SFL_select_1,SFL_select_2,tiles_running,domains_running,trials_running)
tiles_running,domains_running,trials_running = indentify_tiles(SFH_select_1,SFH_select_2,tiles_running,domains_running,trials_running)

Found 9 granules at point {"type": "Point", "coordinates": [-123.63125, 47.89375]} from 2018-09-01 to 2018-09-30


/Users/jpflug/opt/anaconda3/envs/pangeo/lib/python3.8/site-packages/pystac_client/item_search.py:849: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


Found 7 granules at point {"type": "Point", "coordinates": [-105.62291666666667, 40.016666666666666]} from 2018-09-01 to 2018-09-30


/Users/jpflug/opt/anaconda3/envs/pangeo/lib/python3.8/site-packages/pystac_client/item_search.py:849: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


Found 7 granules at point {"type": "Point", "coordinates": [-105.60624999999999, 40.104166666666664]} from 2018-09-01 to 2018-09-30


...

/Users/jpflug/opt/anaconda3/envs/pangeo/lib/python3.8/site-packages/pystac_client/item_search.py:849: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


Found 14 granules at point {"type": "Point", "coordinates": [-118.45625000000001, 36.29374999999999]} from 2018-09-01 to 2018-09-30


/Users/jpflug/opt/anaconda3/envs/pangeo/lib/python3.8/site-packages/pystac_client/item_search.py:849: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


Found 10 granules at point {"type": "Point", "coordinates": [-119.45, 37.65]} from 2018-09-01 to 2018-09-30


/Users/jpflug/opt/anaconda3/envs/pangeo/lib/python3.8/site-packages/pystac_client/item_search.py:849: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


In [69]:
write_txtFile(hls_out+'all_tiles.txt',tiles_running)
write_txtFile(hls_out+'all_domains.txt',domains_running)
write_txtFile(hls_out+'all_trials.txt',trials_running)
unique_tiles = list(set(tiles_running))
write_txtFile(hls_out+'unique_tiles.txt',unique_tiles)

In [68]:
unique_tiles = list(set(tiles_running))
unique_tiles

['10SGH',
 '11SKB',
 '10TGK',
 '10SGG',
 '11SLA',
 '10TFM',
 '10SGJ',
 '10TFL',
 '13SCD',
 '10UDU',
 '13TDE',
 '13SCC',
 '13SCB',
 '10TGM',
 '13SDD',
 '11SKD',
 '10TEL',
 '10TGL']